In [5]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pickle
from sklearn.preprocessing import QuantileTransformer
from sklearn.svm import LinearSVC

In [8]:
train = pickle.load(open(f'../../data/vccfinder/by_time/features_train.pkl', 'rb'))
test = pickle.load(open(f'../../data/vccfinder/by_time/features_test.pkl', 'rb'))
test

,commit_id,LA,LD,LT,FIX,Entropy,NS,ND,NF,NDEV,AGE,NUC,EXP,REXP,SEXP,count_key_word_add,count_key_word_del,count_key,label
0,c5d72beaab1cbbbe68271f4bc4b6670d69985157,72,15,449,1,1.070161,1,1,1,1,114.5,2,127,572.000000,8,29,2,31,0
1,0ff832d31470471803b175cfff4e40c1b08ee779,233,31,8224,0,3.003333,2,2,2,2,46.5,3,5,222.000000,1,63,18,81,0
2,6c573e7f33380e60b583294e9ec5d68ae351c3cf,211,98,10646,0,3.626985,1,1,1,2,4.5,2,3,196.000000,1,46,23,69,1
3,6bb36714528a9563dd3b92720c5063a1284b86f8,7,0,1533,0,0.028768,1,1,1,4,14.0,1,53,45.982143,2,2,0,2,0
4,11cad77da87c4fa2aab7d58dd5339e254db7937e,14,0,565,0,0.102331,1,1,1,3,249.0,1,236,273.255556,5,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4050,0aa576040704401ae28ea73b862d0b5d84262d51,50,2,1832,0,0.166391,2,2,2,3,1728.0,1,419,1015.733333,14,1,1,2,0
4051,8cfe2463cec6835c7b0b73dcb2ab2edaf035e3f9,144,41,366,0,2.040370,2,2,2,0,0.0,1,16,135.000000,3,16,10,26,0
4052,cfb022640b5eec337b06f88a485487dc92ca1ac1,1,1,237,1,0.039339,1,1,1,19,1949.0,1,777,2826.316667,23,0,0,0,0
4053,14f2111f9cec919a450115ce4914f228effca58e,144,184,1922,1,0.851824,2,2,2,0,0.0,1,377,4171.666667,32,12,15,27,0


In [13]:
target = 'label'
features = train.columns.drop(['commit_id', target])
X_train, y_train = train[features], train[target]
X_test, y_test = test[features], test[target]
scaler = QuantileTransformer(n_quantiles=10, output_distribution='uniform')
for col in X_train.columns:
    X_train[col] = scaler.fit_transform(X_train[[col]])
    X_test[col] = scaler.fit_transform(X_test[[col]])
X_train

,LA,LD,LT,FIX,Entropy,NS,ND,NF,NDEV,AGE,NUC,EXP,REXP,SEXP,count_key_word_add,count_key_word_del,count_key
0,0.730994,0.847953,0.667265,0.0,0.892140,0.000000,0.890873,0.000000,0.037037,0.559454,0.000000,0.000000,0.000000,0.000000,0.777778,0.844444,0.782716
1,0.868905,0.889016,0.701026,1.0,0.901908,0.894737,0.890873,0.894737,0.111111,0.487093,0.891253,0.301587,0.450406,0.388889,0.809028,0.800000,0.795062
2,0.792166,0.888964,0.866699,1.0,0.900288,0.000000,0.000000,0.000000,0.611111,0.675319,0.888889,0.111111,0.125604,0.055556,0.890315,0.892240,0.895351
3,0.890315,0.891277,0.889353,1.0,0.904857,0.888889,0.888889,0.888889,0.633333,0.624756,0.903073,0.083333,0.113527,0.111111,0.895753,0.896763,0.909421
4,0.782574,0.583333,0.741624,0.0,0.889483,0.000000,0.894841,0.000000,0.466667,0.510101,0.893617,0.623116,0.737178,0.481481,0.815972,0.666667,0.787654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16214,0.222222,0.000000,0.370003,0.0,0.195199,0.000000,0.000000,0.000000,0.355556,0.259259,0.000000,0.602736,0.785228,0.688889,0.277778,0.000000,0.222222
16215,0.771930,0.277778,0.457528,0.0,0.355535,0.000000,0.888889,0.000000,0.000000,0.000000,0.000000,0.383333,0.586929,0.111111,0.444444,0.000000,0.333333
16216,0.444444,0.555556,0.227830,0.0,0.531300,0.000000,0.000000,0.000000,0.466667,0.333333,0.000000,0.483157,0.635755,0.555556,0.277778,0.500000,0.333333
16217,0.782574,0.839181,0.245005,1.0,0.879011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.533937,0.741594,0.055556,0.744444,0.866667,0.780247


In [70]:
model = LinearSVC(max_iter=200000)
model.fit(X_train, y_train)
y_pred = model._predict_proba_lr(X_test)
y_pred = np.array([y_pred[i][1] for i in range(len(y_pred))])

In [71]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
thresh = 0.5
print(f"Precision:  {precision_score(y_test, y_pred >= thresh):.4f}")
print(f"Recall:  {recall_score(y_test, y_pred >= thresh):.4f}")
print(f"F1 score:  {f1_score(y_test, y_pred >= thresh):.4f}")
print(f"Accuracy:  {accuracy_score(y_test, y_pred >= thresh):.4f}")
print(f"AUC:  {roc_auc_score(y_test, y_pred):.4f}")

Precision:  0.4970
Recall:  0.7506
F1 score:  0.5981
Accuracy:  0.6947
AUC:  0.7841
